In [3]:
##### import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data
from PIL import  Image,ImageDraw
import numpy as np
import random
import torch 

Batch_Size = 50
num_epochs = 200
learning_rate = 1e-5
samples = 20000

data1 = []
data2= []


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% --  Generate dataset


cap = cv2.VideoCapture('video_drone_tank.avi')

uframes = []
ret, frame = cap.read()

uframes.append(frame)
flag = 0
z = 0
while(z<30):
    ret, frame = cap.read()
    #np_frame = cv2.imread('video', frame) # does not work
    #np_frame = np.asarray(cv2.GetMat(frame)) # does not work
    #print(np_frame.shape)
    for a in uframes:
        if (np.array(a).any() != None):
            print("a: ",np.shape(a))
            print("frame: ",np.shape(a))
            difference = cv2.subtract(np.array(a), np.array(frame))    
            result = not np.any(difference)
            #print("inside for")
            if result is True:
                flag = 1
                #print("setting flag")
                break
    if(not flag):
        #print("appending")
        uframes.append(frame)
    z+=1

for i,frame in enumerate(uframes):
    uframes[i] = cv2.resize(frame,(100,100))
    
print(np.shape(uframes[0]))
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
print("length: ",len(uframes))
for i in range(samples):
    data1.append(np.array(np.transpose(np.asarray(uframes[i % len(uframes)]),(2,0,1)),dtype=np.float32))
    data2.append(np.array(np.transpose(np.asarray(uframes[(i+1) % len(uframes)]),(2,0,1)),dtype=np.float32))

    

#%%%%%%%%%%%%%%%%%%%%%%%%%%% Noisy data

im1 = Image.open('tree1.png')
im1 = im1.resize((10,10))

im2 = Image.open('tree2.png')
im2 = im2.resize((10,10))

im3 = Image.open('tree3.png')
im3 = im3.resize((10,10))

img = Image.new('RGB', (100,100), color = '#9b7653')
img.paste(im1,(5,75),im1)
img.paste(im1,(10,55),im1)
img.paste(im1,(25,50),im1)
img.paste(im2,(75,5),im2)
img.paste(im2,(60,30),im2)
img.paste(im2,(90,20),im2)
img.paste(im3,(75,60),im3)
img.paste(im3,(60,80),im3)
im_n = np.reshape(img,(100,100,3))
b = np.asarray(im_n)
b = np.array(b, dtype=np.float32)
b = np.transpose(b,(2,0,1))
b = torch.from_numpy(b)

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%       

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Data Preparation

y0 = np.asarray(data1)
y1 = np.asarray(data2)



print(np.shape(y0))

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)


torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=Batch_Size,

    shuffle=True,

    num_workers=0,

)


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Defining NN

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()

        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(20000, 15)
        self.sigmoid1 = nn.Sigmoid()

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pool 2 
        out = self.maxpool2(out)

        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        #out = self.sigmoid1(out)
        return out
    
model = CNNModel()


optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Loss Function

def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)
    #print(fx, np.shape(fx))
    #print(gy, np.shape(gy))
    #print(np.shape((torch.transpose(fx,3,2))),np.shape(fx))
    covf = torch.matmul(fx.t(), fx) / Nsamples

    covg = torch.matmul(gy.t(), gy) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)





for epoch in range(num_epochs):
    for x,y in loader:
        optimizer.zero_grad()
        # ===================forward=====================
        #loss = criterion(output1, img)
        loss = h_score(model(x),model(y))
        # ===================backward====================

        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print("x: ", model(x[0].reshape(1,3,100, 100)))
    print("y:", model(y[0].reshape(1,3,100, 100)))
    print("b:", model(b.reshape(1,3,100, 100)))


a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 

epoch [3/200], loss:-1.4173
x:  tensor([[  1.9212,  16.4378, -15.2327,  -0.9957,  46.3028,  13.6353,   3.2720,
         -12.8448,  16.9181,  -5.6722,  44.5184, -30.2708,  13.5996,  40.3753,
           8.1478]], grad_fn=<AddmmBackward>)
y: tensor([[  1.7900,  16.8972, -14.9019,  -0.8351,  46.4136,  13.3962,   3.1387,
         -13.0224,  17.0262,  -5.6762,  44.4915, -30.2607,  13.4165,  40.1494,
           8.6673]], grad_fn=<AddmmBackward>)
b: tensor([[  4.6099,  22.5576, -16.9190,  -2.7285,  56.1667,  15.8831,   1.8881,
         -10.0321,  20.2975,  -6.2371,  56.8154, -36.8945,  18.4477,  50.2314,
           9.9120]], grad_fn=<AddmmBackward>)
epoch [4/200], loss:-1.5529
x:  tensor([[  8.8631,  11.5700, -11.0905,  27.0268,  31.0173,  10.2132, -15.4545,
         -14.6222,   8.0524,   1.3476, -13.8741, -13.7640,   0.9692,  19.4870,
          32.2686]], grad_fn=<AddmmBackward>)
y: tensor([[  8.5238,  12.0638, -10.8045,  27.2559,  31.1891,   9.8079, -15.7517,
         -14.8700,   8.2735,   1

epoch [16/200], loss:-1.6520
x:  tensor([[  8.4343, -20.1855,  11.1268,  24.7807,   6.6474,  16.9557,   0.6347,
          -5.3377,   7.7182,  -3.7063,  -4.3341,  -0.3775,  33.5846, -21.8893,
          18.8652]], grad_fn=<AddmmBackward>)
y: tensor([[  8.4639, -20.2978,  11.0308,  24.1660,   6.6840,  16.9200,   0.6753,
          -5.2872,   7.7653,  -3.7086,  -4.3002,  -0.4365,  33.5849, -22.5435,
          18.0255]], grad_fn=<AddmmBackward>)
b: tensor([[ 13.2861, -18.8272,  18.1805,  24.3788,   5.6994,  20.8707,   0.2760,
          -2.5535,   5.1690,  -3.4651,  -3.9025,   2.9293,  35.4708, -24.2801,
          22.7046]], grad_fn=<AddmmBackward>)
epoch [17/200], loss:-1.6721
x:  tensor([[ -2.3090,  -8.6227,   8.7145,  45.0314,   1.6420,  25.4074,   1.5111,
           0.4429,  13.7555,   1.6821,   1.9382,  15.1063,   2.4743, -43.9472,
          -0.5065]], grad_fn=<AddmmBackward>)
y: tensor([[ -2.1806,  -8.4188,   8.9510,  45.0150,   1.4798,  25.5298,   1.7069,
           0.3473,  13.4414,  

epoch [29/200], loss:-1.6731
x:  tensor([[  1.0582, -17.1817, -12.5859,  47.0578,   2.5006,   8.4485,   4.8870,
          -8.8329, -25.0626,   0.5565,   2.7614,  -9.6404, -10.4797, -26.5731,
         -21.0847]], grad_fn=<AddmmBackward>)
y: tensor([[  1.1405, -16.8649, -12.3040,  47.0290,   2.3113,   8.6098,   5.0369,
          -8.9552, -25.4052,   0.5360,   2.6511,  -9.3268, -10.9085, -25.9530,
         -20.9786]], grad_fn=<AddmmBackward>)
b: tensor([[  3.8662, -15.3884, -10.0679,  50.7352,   0.5831,   9.9171,   4.8850,
          -7.0289, -32.9300,   1.6571,   4.3119,  -8.3454, -14.7140, -28.0119,
         -22.3469]], grad_fn=<AddmmBackward>)
epoch [30/200], loss:-1.2831
x:  tensor([[-10.5023,  21.5271, -17.5299,  21.2478,  22.5795, -15.2482,  -1.6558,
         -11.4967,  -5.2267,  -0.3069,  -8.4434,   2.3547,   1.6424, -45.7367,
         -11.9165]], grad_fn=<AddmmBackward>)
y: tensor([[-10.5647,  21.1265, -17.8773,  21.0266,  22.7231, -15.3486,  -1.7328,
         -11.4291,  -4.7929,  

epoch [42/200], loss:-1.5196
x:  tensor([[  1.4650,   5.4547,  -4.9102,  -8.6463,  11.0878,   9.3893,  -4.9833,
          -6.4275, -23.6777,   3.9436, -18.0994,   7.9979,  32.4826, -28.4489,
          38.6840]], grad_fn=<AddmmBackward>)
y: tensor([[  1.5640,   5.8669,  -4.5545,  -8.7953,  10.8818,   9.5127,  -4.8724,
          -6.5242, -24.1052,   3.9210, -18.2243,   8.3897,  31.9695, -27.8057,
          38.8417]], grad_fn=<AddmmBackward>)
b: tensor([[  4.0070,  11.3008,  -1.1674, -12.8702,  10.3115,  11.1032,  -6.4144,
          -4.4482, -31.1550,   5.4704, -19.7072,  12.4468,  35.2270, -29.5903,
          46.2238]], grad_fn=<AddmmBackward>)
epoch [43/200], loss:-1.3542
x:  tensor([[  1.8012,  10.7950,  -4.6840,  18.2110,  16.5838,  12.7129,   5.5070,
         -11.2889, -16.6919,  -2.4035,   2.1432, -11.2084,  19.1272, -12.3919,
          29.8327]], grad_fn=<AddmmBackward>)
y: tensor([[  1.8593,  10.5668,  -4.9224,  16.4957,  16.6115,  12.6657,   5.4720,
         -11.2706, -16.5173,  

...

epoch [146/200], loss:-1.4422
x:  tensor([[ -2.5530,  -0.6768,  -4.7619,  22.9930,   8.0685,   7.1725,  -1.6687,
          -4.1600,  -1.9790,   4.7939,  -4.9409,  -4.6897,  -9.0873, -34.5120,
           6.5712]], grad_fn=<AddmmBackward>)
y: tensor([[ -2.5521,  -0.3011,  -4.4086,  23.0265,   7.9696,   7.2365,  -1.6437,
          -4.1863,  -2.3783,   4.7849,  -4.9459,  -4.3206,  -9.6556, -34.5687,
           6.1605]], grad_fn=<AddmmBackward>)
b: tensor([[ -1.9464,   2.4518,  -0.6653,  22.4684,   7.5374,   6.8357,  -2.6653,
          -1.9672,  -4.6334,   5.9705,  -3.6030,  -2.7621, -11.5934, -34.5365,
           6.6707]], grad_fn=<AddmmBackward>)
epoch [147/200], loss:-1.4579
x:  tensor([[ -1.3620,   1.6236, -16.1204,  20.4727,   5.1968,  10.8391,   1.7843,
          -6.1314,  -4.0606,   2.5041,  -2.5256,   2.5719,  -2.5443, -18.5557,
           0.9286]], grad_fn=<AddmmBackward>)
y: tensor([[ -1.3567,   1.5470, -16.2056,  21.7036,   5.2489,  10.8420,   1.7879,
          -6.1330,  -4.0127,

epoch [159/200], loss:-1.3533
x:  tensor([[ -2.6378,  -0.6565, -13.7506,  40.2601,   3.1162,   7.8769,  -0.0499,
          -5.3270, -11.1740,   2.4849,  -2.9680,  10.2670,  -7.7038,  -6.9223,
         -12.0079]], grad_fn=<AddmmBackward>)
y: tensor([[ -2.6445,  -0.8075, -13.8711,  40.7381,   3.1524,   7.8558,  -0.0740,
          -5.3449, -11.0051,   2.4694,  -2.9690,  10.1063,  -7.5224,  -8.0630,
         -11.8665]], grad_fn=<AddmmBackward>)
b: tensor([[ -2.0141,   2.2929, -10.2194,  41.8379,   2.3204,   7.4838,  -0.8861,
          -3.3760, -13.6940,   3.4894,  -1.5550,  12.7723,  -9.6978,  -7.2278,
         -12.1164]], grad_fn=<AddmmBackward>)
epoch [160/200], loss:-1.4545
x:  tensor([[ -2.6864,  -4.4655,   6.1354,  35.5216,   5.8204,   6.3846,  -1.1552,
          -4.7976, -11.6071,   3.4299,  -2.8682,   6.4020,   2.6847, -18.0200,
           5.9214]], grad_fn=<AddmmBackward>)
y: tensor([[ -2.6526,  -4.6230,   5.9698,  34.1537,   5.7987,   6.3603,  -1.1537,
          -4.8091, -11.4447,

epoch [172/200], loss:-1.4544
x:  tensor([[ -3.4395, -13.9430,  -9.8018,  27.7259,   6.6685,   5.4314,  -0.5776,
          -7.4425, -11.4532,   3.1344,  -3.5275,  -2.8163,  -9.8141, -12.1081,
           2.3908]], grad_fn=<AddmmBackward>)
y: tensor([[ -3.4498, -14.1202,  -9.9646,  26.3636,   6.6988,   5.4200,  -0.5789,
          -7.4083, -11.3161,   3.1290,  -3.5262,  -3.0070,  -9.4613, -11.5183,
           3.7228]], grad_fn=<AddmmBackward>)
b: tensor([[ -2.8650, -11.8427,  -6.2672,  27.2017,   6.3451,   4.9128,  -1.2598,
          -5.6520, -13.6368,   4.0164,  -2.4097,  -1.2202, -11.4584, -12.9347,
           4.4909]], grad_fn=<AddmmBackward>)
epoch [173/200], loss:-1.6173
x:  tensor([[ -0.5794,   1.6118,  -6.9550,  14.7239,   8.1031,   8.7323,   0.3677,
          -5.1885, -12.1938,   4.2157,  -2.9918,   3.3126,  -7.7656, -37.8411,
         -11.7705]], grad_fn=<AddmmBackward>)
y: tensor([[ -0.5368,   1.5374,  -7.0225,  13.8407,   8.0813,   8.6917,   0.3547,
          -5.1845, -12.1249,

epoch [185/200], loss:-1.6297
x:  tensor([[ -1.5360,  -0.7314,   0.0578,   8.4344,   4.2263,   5.4131,  -1.4737,
          -5.1972,  -4.2790,   2.9797,  -3.4147,   2.3340, -13.2259,  -4.1902,
          14.6000]], grad_fn=<AddmmBackward>)
y: tensor([[ -1.5350,  -1.1748,  -0.3442,   9.0489,   4.2610,   5.4013,  -1.4793,
          -5.1924,  -3.8199,   2.9850,  -3.4129,   1.9050, -12.5946,  -3.7005,
          14.6136]], grad_fn=<AddmmBackward>)
b: tensor([[ -0.6984,   1.4182,   3.9897,   6.0697,   3.8385,   4.7614,  -2.1294,
          -3.2215,  -5.8190,   3.8792,  -2.2246,   3.7139, -14.2636,  -3.2043,
          14.8850]], grad_fn=<AddmmBackward>)
epoch [186/200], loss:-1.4793
x:  tensor([[ -0.5284,  11.3381,  -7.9223,  13.0412,   8.9934,   6.1726,  -0.7254,
         -11.6560,  -1.9424,   3.5030,  -2.3985,  -7.2636, -14.5615,  -4.5035,
          22.5237]], grad_fn=<AddmmBackward>)
y: tensor([[ -0.5308,  11.3389,  -7.9460,  14.2847,   8.9871,   6.1664,  -0.7272,
         -11.6525,  -1.9438,

epoch [198/200], loss:-1.5297
x:  tensor([[ -1.6235,  -3.3433,  -2.6466,  24.8546,   5.8105,   6.2188,  -1.2249,
          -4.9622,  -1.3377,   3.2874,  -0.8940,  -8.4169, -10.5743,  -2.4431,
           4.5046]], grad_fn=<AddmmBackward>)
y: tensor([[ -1.6373,  -3.3960,  -2.7117,  26.1443,   5.8407,   6.2123,  -1.2188,
          -4.9442,  -1.2954,   3.2845,  -0.8665,  -8.4675, -10.5413,  -1.6365,
           4.7654]], grad_fn=<AddmmBackward>)
b: tensor([[ -0.8141,  -1.2067,   0.7974,  24.4630,   5.4668,   5.4384,  -1.7372,
          -3.0639,  -2.7865,   4.0837,   0.3652,  -7.1202, -11.6363,  -1.2310,
           5.0338]], grad_fn=<AddmmBackward>)
epoch [199/200], loss:-1.5487
x:  tensor([[-2.0375,  1.2828, -1.4292, 28.0819,  4.9134,  5.8055, -1.5963, -4.8343,
         -3.3120,  4.3580, -3.4972,  2.6189, -1.4911,  0.2008,  2.1165]],
       grad_fn=<AddmmBackward>)
y: tensor([[-2.0307e+00,  1.2444e+00, -1.4530e+00,  2.6980e+01,  4.9161e+00,
          5.8192e+00, -1.6131e+00, -4.8365e+00, -3

In [3]:
data3=[]

class_data = []

for i in range(75):
    img = uframes[0]
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    #draw = ImageDraw.Draw(img)
    #draw.ellipse((50,50,55,55), fill=(0, 0, 255))
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),0))



In [4]:
data3=[]

for i in range(75):
    img = uframes[1]

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),2))
    
print(len(class_data))

150


In [5]:
data3=[]

for i in range(75):
    img = uframes[2]
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),3))
    
print(len(class_data))

225


In [3]:
import random


p=0
class_data = []
while(p<len(uframes)):
    data3=[]
    print(p)
    for i in range(50):
        img = uframes[p]
        data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
        y3 = np.array(data3)
        x_1 = torch.from_numpy(y3)
        temp = model(x_1[i].reshape(1,3,100,100))
        #print(temp)
        class_data.append((list(temp.detach().numpy()[0]),p))
    p+=1
    print(len(class_data))

random.shuffle(class_data)
random.shuffle(class_data)

print("length ", len(class_data))

NameError: name 'uframes' is not defined

In [6]:
data3=[]

for i in range(75):
    img = uframes[3]

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    class_data.append((list(temp.detach().numpy()[0]),1))
    
print(len(class_data))

300


In [18]:

cap = cv2.VideoCapture('video_drone_tank.avi')

uframes = []
ret, frame = cap.read()

uframes.append(frame)
flag = 0
z = 0
while(z<30):
    ret, frame = cap.read()
    #np_frame = cv2.imread('video', frame) # does not work
    #np_frame = np.asarray(cv2.GetMat(frame)) # does not work
    #print(np_frame.shape)
    for a in uframes:
        if (np.array(a).any() != None):
            print("a: ",np.shape(a))
            print("frame: ",np.shape(a))
            difference = cv2.subtract(np.array(a), np.array(frame))    
            result = not np.any(difference)
            #print("inside for")
            if result is True:
                flag = 1
                #print("setting flag")
                break
    if(not flag):
        #print("appending")
        uframes.append(frame)
    z+=1

frame = Image.fromarray(uframes[0])


video = cv2.VideoWriter('video_test.avi', 0, 1, (720,1080))

for image in uframes:
    video.write(image)


for i,frame in enumerate(uframes):
    uframes[i] = cv2.resize(frame,(100,100))
cv2.destroyAllWindows()
video.release()

a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 720, 3)
a:  (1080, 720, 3)
frame:  (1080, 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from colorama import init 
from termcolor import colored 
import time



#z  = np.array(np.transpose(np.asarray(z),(2,0,1)),dtype=np.float32)
# import some data to play with
print("Length of Class data: ", len(class_data))
# Take the first two features. We could avoid this by using a two-dim dataset
X = [(i[0]) for i in class_data]
y = [i[1] for i in class_data]
#print(X,np.shape(X))
#print(y,np.shape(y))
# we create an instance of SVM and fit out data. We do not scale our
# data since we want to plot the support vectors
C = 1.0  # SVM regularization parameter
clf = svm.SVC(kernel='linear', C=C)
clf = clf.fit(X, y)
data3=[]
y3=0

n1 = "\x1b[31m\"Drone Stationary, Tank Firing\"\x1b[0m"
n2 = "\x1b[31m\"Drone Reloading, Tank Firing\"\x1b[0m"
n3 = "\x1b[31m\"Drone Firing, Tank Stationary\"\x1b[0m"
n4 = "\x1b[31m\"Drone Firing, Tank Reloading\"\x1b[0m"
n5 = "\x1b[31m\"Drone Firing, Tank Firing\"\x1b[0m"

names = ['Just Background','Only Tank moving', 'Only Drone Moving', 'Drone stationary, Tank Stationary', 'Drone Stationary, Tank Reloading' ,
         n1, 'Drone reloading, Tank Stationary', 'Drone reloading, Tank Reloading', n2 , n3,n4,n5]

print("Start")
for i in range(2):
    z = cv2.imread('Images1/4a.png')
    z  = cv2.resize(z,(100,100))
    img = z
    print(np.shape(img))
    #ms = Image.fromarray(z)
    #ms.show()
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))
    temp = temp.detach().numpy()
    #print(temp)
    z = clf.predict(temp)
    print(names[z[0]])
    


NameError: name 'class_data' is not defined

In [54]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    #draw = ImageDraw.Draw(img)
    #draw.ellipse((50,50,55,55), fill=(0, 0, 255))
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))
    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   24.159393    4.10358     6.0005293
   -0.6879165  -2.862772   -5.096247    4.1649837  -2.2589116  -1.0829961
  -10.358628   -9.089189    4.2659616]]
[5]
[[ -1.5474473   3.4351084   5.324524   2

In [55]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(5,5),im_drone)
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

NameError: name 'im_drone' is not defined

In [53]:
data3=[]
for i in range(20):
    img = Image.new('RGB', (100, 100), color = '#9b7653')
    img.paste(im1,(5,75),im1)
    img.paste(im1,(10,55),im1)
    img.paste(im1,(25,50),im1)
    img.paste(im2,(75,5),im2)
    img.paste(im2,(60,30),im2)
    img.paste(im2,(90,20),im2)
    img.paste(im3,(75,60),im3)
    img.paste(im3,(60,80),im3)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(25,80),im_drone)
    #img.paste(im_drone,(40,40),im_drone)
    #img.paste(imb,(80,80),imb)
    img.paste(im_drone,(72,65),im_drone)
    img.paste(imb,(80,80),imb)
    #img.show()
    #im2_pos1 = np.reshape(im2_pos1,(100,100,3))

    data3.append(np.array(np.transpose(np.asarray(img),(2,0,1)),dtype=np.float32))
    y3 = np.array(data3)
    x_1 = torch.from_numpy(y3)
    temp = model(x_1[i].reshape(1,3,100,100))

    temp = temp.detach().numpy()
   
    print(temp)
    z = clf.predict(temp)
    print(z)

NameError: name 'im_drone' is not defined

In [2]:
img = np.transpose(data1[5],(1,2,0))
print(np.shape(np.transpose(data1[5],(1,2,0))))
cv2.imshow('test', img)

img = data2[0]
ms = Image.fromarray(img)
ms.show()

(100, 100, 3)


TypeError: Cannot handle this data type: (1, 1, 100), <f4